In [1]:
import pandas as pd
from uncertainties import ufloat, unumpy
from uncertainties.umath import log10
import numpy as np

# Asegurarse de que tienes las columnas en tu DataFrame
# df['H_ALPHA_FLUX'], df['H_ALPHA_FLUX_ERR'], df['OIII_5007_FLUX'], df['OIII_5007_FLUX_ERR'],
# df['H_BETA_FLUX'], df['H_BETA_FLUX_ERR'], df['NII_6584_FLUX'], df['NII_6584_FLUX_ERR']

df = pd.read_csv('measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])

# Crear columnas con ufloat
df['H_ALPHA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_ALPHA_FLUX'], row['H_ALPHA_FLUX_ERR']), axis=1)
df['OIII_5007_FLUX_UN'] = df.apply(lambda row: ufloat(row['OIII_5007_FLUX'], row['OIII_5007_FLUX_ERR']), axis=1)
df['H_BETA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_BETA_FLUX'], row['H_BETA_FLUX_ERR']), axis=1)
df['NII_6584_FLUX_UN'] = df.apply(lambda row: ufloat(row['NII_6584_FLUX'], row['NII_6584_FLUX_ERR']), axis=1)

# Calcular las razones con incertidumbres
o3_hb_un = df['OIII_5007_FLUX_UN'] / df['H_BETA_FLUX_UN']
nii_ha_un = df['NII_6584_FLUX_UN'] / df['H_ALPHA_FLUX_UN']

# Calcular O3N2 = log10((OIII 5007 / Hbeta) / (NII 6584 / Halpha))
o3n2_un = unumpy.log10(o3_hb_un / nii_ha_un)

# Definir constantes con incertidumbres
const_o3n2 = ufloat(8.533, 0.012)
coef_o3n2 = ufloat(0.214, 0.012)

# Calcular 12 + log(O/H) usando O3N2
abund_o3n2_un = const_o3n2 - coef_o3n2 * o3n2_un

# Calcular N2 = log10(NII 6584 / Halpha)
n2_un = unumpy.log10(nii_ha_un)

# Definir constantes con incertidumbres
const_n2 = ufloat(8.743, 0.027)
coef_n2 = ufloat(0.462, 0.024)

# Calcular 12 + log(O/H) usando N2
abund_n2_un = const_n2 + coef_n2 * n2_un

# Añadir las columnas calculadas al DataFrame
df['O3N2'] = unumpy.nominal_values(o3n2_un)
df['O3N2_ERR'] = unumpy.std_devs(o3n2_un)
df['12_LOG_OH_O3N2'] = unumpy.nominal_values(abund_o3n2_un)
df['12_LOG_OH_O3N2_ERR'] = unumpy.std_devs(abund_o3n2_un)

df['N2'] = unumpy.nominal_values(n2_un)
df['N2_ERR'] = unumpy.std_devs(n2_un)
df['12_LOG_OH_N2'] = unumpy.nominal_values(abund_n2_un)
df['12_LOG_OH_N2_ERR'] = unumpy.std_devs(abund_n2_un)

# Mostrar el DataFrame con las nuevas columnas
print(df[['O3N2', 'O3N2_ERR', '12_LOG_OH_O3N2', '12_LOG_OH_O3N2_ERR', 'N2', 'N2_ERR', '12_LOG_OH_N2', '12_LOG_OH_N2_ERR']])


       O3N2  O3N2_ERR  12_LOG_OH_O3N2  12_LOG_OH_O3N2_ERR        N2    N2_ERR  \
0  2.301985  0.112996        8.040375            0.038624 -1.558421  0.111177   
1  2.154682  0.126795        8.071898            0.039355 -1.457303  0.123748   
2  1.655616  0.063629        8.178698            0.026910 -1.185229  0.058070   
3  2.027372  0.247328        8.099142            0.059475 -1.391182  0.246533   
4  1.691130  0.173196        8.171098            0.043927 -1.270309  0.172368   
5  1.628656  0.074348        8.184468            0.027912 -1.202156  0.071916   
6  2.106784  0.080028        8.082148            0.032809 -1.469700  0.075061   
7  1.362967  0.120515        8.241325            0.032812 -1.130775  0.117968   

   12_LOG_OH_N2  12_LOG_OH_N2_ERR  
0      8.023009          0.069037  
1      8.069726          0.072256  
2      8.195424          0.047517  
3      8.100274          0.121723  
4      8.156117          0.089443  
5      8.187604          0.051627  
6      8.063999   

In [2]:
import pandas as pd
from uncertainties import ufloat, unumpy
from uncertainties.umath import log10

df = pd.read_csv('measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])

# Crear columnas con ufloat en tu DataFrame
df['H_ALPHA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_ALPHA_FLUX'], row['H_ALPHA_FLUX_ERR']), axis=1)
df['OIII_5007_FLUX_UN'] = df.apply(lambda row: ufloat(row['OIII_5007_FLUX'], row['OIII_5007_FLUX_ERR']), axis=1)
df['H_BETA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_BETA_FLUX'], row['H_BETA_FLUX_ERR']), axis=1)
df['NII_6584_FLUX_UN'] = df.apply(lambda row: ufloat(row['NII_6584_FLUX'], row['NII_6584_FLUX_ERR']), axis=1)

# Calcular las razones con incertidumbres
df['OIII_HB_UN'] = df.apply(lambda row: row['OIII_5007_FLUX_UN'] / row['H_BETA_FLUX_UN'], axis=1)
df['NII_HA_UN'] = df.apply(lambda row: row['NII_6584_FLUX_UN'] / row['H_ALPHA_FLUX_UN'], axis=1)

# Calcular O3N2 = log10((OIII 5007 / Hbeta) / (NII 6584 / Halpha))
df['O3N2_UN'] = df.apply(lambda row: log10(row['OIII_HB_UN'] / row['NII_HA_UN']), axis=1)

# Definir constantes con incertidumbres para O3N2
const_o3n2 = ufloat(8.533, 0.012)
coef_o3n2 = ufloat(0.214, 0.012)

# Calcular 12 + log(O/H) usando O3N2
df['12_LOG_OH_O3N2_UN'] = df.apply(lambda row: const_o3n2 - coef_o3n2 * row['O3N2_UN'], axis=1)

# Calcular N2 = log10(NII 6584 / Halpha)
df['N2_UN'] = df['NII_HA_UN'].apply(log10)

# Definir constantes con incertidumbres para N2
const_n2 = ufloat(8.743, 0.027)
coef_n2 = ufloat(0.462, 0.024)

# Calcular 12 + log(O/H) usando N2
df['12_LOG_OH_N2_UN'] = df.apply(lambda row: const_n2 + coef_n2 * row['N2_UN'], axis=1)

# Añadir las columnas calculadas al DataFrame
df['O3N2'] = df['O3N2_UN'].apply(unumpy.nominal_values)
df['O3N2_ERR'] = df['O3N2_UN'].apply(unumpy.std_devs)
df['12_LOG_OH_O3N2'] = df['12_LOG_OH_O3N2_UN'].apply(unumpy.nominal_values)
df['12_LOG_OH_O3N2_ERR'] = df['12_LOG_OH_O3N2_UN'].apply(unumpy.std_devs)

df['N2'] = df['N2_UN'].apply(unumpy.nominal_values)
df['N2_ERR'] = df['N2_UN'].apply(unumpy.std_devs)
df['12_LOG_OH_N2'] = df['12_LOG_OH_N2_UN'].apply(unumpy.nominal_values)
df['12_LOG_OH_N2_ERR'] = df['12_LOG_OH_N2_UN'].apply(unumpy.std_devs)

# Mostrar el DataFrame con las nuevas columnas
print(df[['O3N2', 'O3N2_ERR', '12_LOG_OH_O3N2', '12_LOG_OH_O3N2_ERR', 'N2', 'N2_ERR', '12_LOG_OH_N2', '12_LOG_OH_N2_ERR']])


       O3N2  O3N2_ERR  12_LOG_OH_O3N2  12_LOG_OH_O3N2_ERR        N2    N2_ERR  \
0  2.301985  0.112996        8.040375            0.038624 -1.558421  0.111177   
1  2.154682  0.126795        8.071898            0.039355 -1.457303  0.123748   
2  1.655616  0.063629        8.178698            0.026910 -1.185229  0.058070   
3  2.027372  0.247328        8.099142            0.059475 -1.391182  0.246533   
4  1.691130  0.173196        8.171098            0.043927 -1.270309  0.172368   
5  1.628656  0.074348        8.184468            0.027912 -1.202156  0.071916   
6  2.106784  0.080028        8.082148            0.032809 -1.469700  0.075061   
7  1.362967  0.120515        8.241325            0.032812 -1.130775  0.117968   

   12_LOG_OH_N2  12_LOG_OH_N2_ERR  
0      8.023009          0.069037  
1      8.069726          0.072256  
2      8.195424          0.047517  
3      8.100274          0.121723  
4      8.156117          0.089443  
5      8.187604          0.051627  
6      8.063999   

In [3]:
import pandas as pd
from uncertainties import ufloat, unumpy

df = pd.read_csv('measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])


# Crear columnas con ufloat en tu DataFrame
df['H_ALPHA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_ALPHA_FLUX'], row['H_ALPHA_FLUX_ERR']), axis=1)
df['OIII_5007_FLUX_UN'] = df.apply(lambda row: ufloat(row['OIII_5007_FLUX'], row['OIII_5007_FLUX_ERR']), axis=1)
df['H_BETA_FLUX_UN'] = df.apply(lambda row: ufloat(row['H_BETA_FLUX'], row['H_BETA_FLUX_ERR']), axis=1)
df['NII_6584_FLUX_UN'] = df.apply(lambda row: ufloat(row['NII_6584_FLUX'], row['NII_6584_FLUX_ERR']), axis=1)

# Calcular las razones con incertidumbres
df['OIII_HB_UN'] = df['OIII_5007_FLUX_UN'] / df['H_BETA_FLUX_UN']
df['NII_HA_UN'] = df['NII_6584_FLUX_UN'] / df['H_ALPHA_FLUX_UN']

# Calcular O3N2 = log10((OIII 5007 / Hbeta) / (NII 6584 / Halpha))
df['O3N2_UN'] = unumpy.log10(df['OIII_HB_UN'] / df['NII_HA_UN'])

# Definir constantes con incertidumbres para O3N2
const_o3n2 = ufloat(8.533, 0.012)
coef_o3n2 = ufloat(0.214, 0.012)

# Calcular 12 + log(O/H) usando O3N2
df['12_LOG_OH_O3N2_UN'] = const_o3n2 - coef_o3n2 * df['O3N2_UN']

# Calcular N2 = log10(NII 6584 / Halpha)
df['N2_UN'] = unumpy.log10(df['NII_HA_UN'])

# Definir constantes con incertidumbres para N2
const_n2 = ufloat(8.743, 0.027)
coef_n2 = ufloat(0.462, 0.024)

# Calcular 12 + log(O/H) usando N2
df['12_LOG_OH_N2_UN'] = const_n2 + coef_n2 * df['N2_UN']

# Añadir las columnas calculadas al DataFrame
df['O3N2'] = unumpy.nominal_values(df['O3N2_UN'])
df['O3N2_ERR'] = unumpy.std_devs(df['O3N2_UN'])
df['12_LOG_OH_O3N2'] = unumpy.nominal_values(df['12_LOG_OH_O3N2_UN'])
df['12_LOG_OH_O3N2_ERR'] = unumpy.std_devs(df['12_LOG_OH_O3N2_UN'])

df['N2'] = unumpy.nominal_values(df['N2_UN'])
df['N2_ERR'] = unumpy.std_devs(df['N2_UN'])
df['12_LOG_OH_N2'] = unumpy.nominal_values(df['12_LOG_OH_N2_UN'])
df['12_LOG_OH_N2_ERR'] = unumpy.std_devs(df['12_LOG_OH_N2_UN'])

# Mostrar el DataFrame con las nuevas columnas
print(df[['O3N2', 'O3N2_ERR', '12_LOG_OH_O3N2', '12_LOG_OH_O3N2_ERR', 'N2', 'N2_ERR', '12_LOG_OH_N2', '12_LOG_OH_N2_ERR']])


       O3N2  O3N2_ERR  12_LOG_OH_O3N2  12_LOG_OH_O3N2_ERR        N2    N2_ERR  \
0  2.301985  0.112996        8.040375            0.038624 -1.558421  0.111177   
1  2.154682  0.126795        8.071898            0.039355 -1.457303  0.123748   
2  1.655616  0.063629        8.178698            0.026910 -1.185229  0.058070   
3  2.027372  0.247328        8.099142            0.059475 -1.391182  0.246533   
4  1.691130  0.173196        8.171098            0.043927 -1.270309  0.172368   
5  1.628656  0.074348        8.184468            0.027912 -1.202156  0.071916   
6  2.106784  0.080028        8.082148            0.032809 -1.469700  0.075061   
7  1.362967  0.120515        8.241325            0.032812 -1.130775  0.117968   

   12_LOG_OH_N2  12_LOG_OH_N2_ERR  
0      8.023009          0.069037  
1      8.069726          0.072256  
2      8.195424          0.047517  
3      8.100274          0.121723  
4      8.156117          0.089443  
5      8.187604          0.051627  
6      8.063999   

In [13]:
import pandas as pd
from uncertainties import ufloat, unumpy

# Supongamos que ya tienes un DataFrame llamado df con las siguientes columnas:
# 'SII_6716_FLUX', 'SII_6716_FLUX_ERR', 'SII_6731_FLUX', 'SII_6731_FLUX_ERR'

df = pd.read_csv('txt/measurement_emission_lines.tsv', delimiter='\t')
df = df.drop(index=[8,9,10,11,12])

# Crear columnas de ufloat para los flujos
df['SII_6716_FLUX_UN'] = df.apply(lambda row: ufloat(row['SII_6716_FLUX'], row['SII_6716_FLUX_ERR']), axis=1)
df['SII_6731_FLUX_UN'] = df.apply(lambda row: ufloat(row['SII_6731_FLUX'], row['SII_6731_FLUX_ERR']), axis=1)

# Calcular la razón de [S II] 6716 / 6731 con incertidumbres
df['RSII'] = df['SII_6716_FLUX_UN'] / df['SII_6731_FLUX_UN']

# Añadir los resultados al DataFrame
df['RS2'] = unumpy.nominal_values(df['RSII'])
df['RS2_ERR'] = unumpy.std_devs(df['RSII'])

print(df[['RS2','RS2_ERR']])


        RS2   RS2_ERR
0  1.300890  0.029984
1  1.345630  0.027156
2  1.570315  0.095763
3  1.306804  0.024645
4  1.314939  0.023713
5  1.416639  0.044546
6  1.418157  0.032516
7  1.278280  0.051656


In [7]:
df['RS2']

0    1.300890
1    1.345630
2    1.570315
3    1.306804
4    1.314939
5    1.416639
6    1.418157
7    1.278280
Name: RS2, dtype: float64

In [8]:
print(df['SII_6716_FLUX']), print(df['SII_6731_FLUX'])

0    6.986487
1    2.082050
2    2.686583
3    3.322156
4    7.358254
5    1.848934
6    0.775210
7    2.251778
Name: SII_6716_FLUX, dtype: float64
0    5.370542
1    1.547268
2    1.710856
3    2.542199
4    5.595892
5    1.305155
6    0.546632
7    1.761569
Name: SII_6731_FLUX, dtype: float64


(None, None)

In [14]:
import pandas as pd
from uncertainties import ufloat, unumpy
import pyneb as pn

# Crear una instancia del objeto Atom para el azufre ionizado
sii = pn.Atom('S', 2)

# Supongamos una temperatura electrónica de 10,000 K
Te = 10000

# Crear columnas de ufloat para RS2 y RS2_ERR
df['RS2_UN'] = df.apply(lambda row: ufloat(row['RS2'], row['RS2_ERR']), axis=1)

# Crear listas para almacenar los valores de densidad electrónica y sus incertidumbres
ne_list = []
ne_err_list = []

for rs2_un in df['RS2_UN']:
    # Obtener el valor nominal y la incertidumbre de RS2
    rs2_nominal = rs2_un.nominal_value
    rs2_err = rs2_un.std_dev

    # Calcular la densidad electrónica y su incertidumbre
    ne = sii.getTemDen(int_ratio=rs2_nominal, tem=Te, wave1=6716, wave2=6731)
    ne_upper = sii.getTemDen(int_ratio=rs2_nominal + rs2_err, tem=Te, wave1=6716, wave2=6731)
    ne_lower = sii.getTemDen(int_ratio=rs2_nominal - rs2_err, tem=Te, wave1=6716, wave2=6731)
    ne_err = (ne_upper - ne_lower) / 2

    # Añadir los valores a las listas
    ne_list.append(ne)
    ne_err_list.append(ne_err)

# Añadir los resultados al DataFrame
df['NE'] = ne_list
df['NE_ERR'] = ne_err_list

print(df)


   ID  ID_MASK  H_BETA_FLUX  H_BETA_FLUX_ERR  OIII_5007_FLUX  \
0   1      1.0    88.649703         2.494409      491.180987   
1   3      3.0    17.124042         0.715287       85.307068   
2   7     11.0    13.285319         0.569428       39.242761   
3   9     13.0    31.443495         0.851705      136.057021   
4  11     15.0    47.815526         1.320968      126.005902   
5  17     21.0    11.200002         0.338545       29.903199   
6  19     23.0     7.826941         0.425261       33.937198   
7  21     25.0     9.668545         0.342895       16.502650   

   OIII_5007_FLUX_ERR  H_ALPHA_FLUX  H_ALPHA_FLUX_ERR  NII_6584_FLUX  \
0           18.179230    280.129860          2.023057       7.743517   
1            4.093336     49.981168          0.499074       1.743826   
2            1.641625     41.935169          0.387233       2.737472   
3            4.998214     90.248278          1.855688       3.666542   
4            3.462590    154.922381          3.912832       8.3

In [3]:
import pyneb as pn

sii = pn.Atom('S', 2)
Te = 10000
ne = sii.getTemDen(int_ratio=1.3, tem=Te, wave1=6716, wave2=6731)

print(f"Densidad electrónica: {ne} cm^-3")


Densidad electrónica: 157.53754255768436 cm^-3


In [16]:
import pandas as pd
from uncertainties import ufloat
import pyneb as pn

# Crear una instancia del objeto Atom para el azufre ionizado
sii = pn.Atom('S', 2)

# Supongamos una temperatura electrónica de 10,000 K
Te = 10000


# Crear columnas de ufloat para RS2 y RS2_ERR
df['RS2_UN'] = df.apply(lambda row: ufloat(row['RS2'], row['RS2_ERR']), axis=1)

# Crear listas para almacenar los valores de densidad electrónica y sus incertidumbres
ne_list = []
ne_err_list = []

for index, row in df.iterrows():
    rs2_un = row['RS2_UN']
    rs2_nominal = rs2_un.nominal_value
    rs2_err = rs2_un.std_dev
    
    if 0.5 <= rs2_nominal <= 1.5:
        try:
            # Calcular densidad electrónica
            ne = sii.getTemDen(int_ratio=rs2_nominal, tem=Te, wave1=6716, wave2=6731)
            
            # Calcular los límites superior e inferior de la densidad electrónica
            ne_upper = sii.getTemDen(int_ratio=rs2_nominal + rs2_err, tem=Te, wave1=6716, wave2=6731)
            ne_lower = sii.getTemDen(int_ratio=rs2_nominal - rs2_err, tem=Te, wave1=6716, wave2=6731)
            
            # Calcular la incertidumbre como la mitad de la diferencia entre los límites superior e inferior
            ne_err = (ne_upper - ne_lower) / 2

            # Añadir los valores a las listas
            ne_list.append(ne)
            ne_err_list.append(ne_err)
        except Exception as e:
            ne_list.append(float('nan'))
            ne_err_list.append(float('nan'))
            print(f"Error calculating ne for index {index}: {e}")
    else:
        ne_list.append(float('nan'))
        ne_err_list.append(float('nan'))
        print(f"Valor de int_ratio {rs2_nominal} fuera del rango válido. No se puede calcular la densidad electrónica.")

# Añadir los resultados al DataFrame
df['NE'] = ne_list
df['NE_ERR'] = ne_err_list

print(df)


Valor de int_ratio 1.5703150937308576 fuera del rango válido. No se puede calcular la densidad electrónica.
   ID  ID_MASK  H_BETA_FLUX  H_BETA_FLUX_ERR  OIII_5007_FLUX  \
0   1      1.0    88.649703         2.494409      491.180987   
1   3      3.0    17.124042         0.715287       85.307068   
2   7     11.0    13.285319         0.569428       39.242761   
3   9     13.0    31.443495         0.851705      136.057021   
4  11     15.0    47.815526         1.320968      126.005902   
5  17     21.0    11.200002         0.338545       29.903199   
6  19     23.0     7.826941         0.425261       33.937198   
7  21     25.0     9.668545         0.342895       16.502650   

   OIII_5007_FLUX_ERR  H_ALPHA_FLUX  H_ALPHA_FLUX_ERR  NII_6584_FLUX  \
0           18.179230    280.129860          2.023057       7.743517   
1            4.093336     49.981168          0.499074       1.743826   
2            1.641625     41.935169          0.387233       2.737472   
3            4.998214     9

0   1.300890    0.029984
1   1.345630    0.027156
2   1.570315    0.095763
3   1.306804    0.024645
4   1.314939    0.023713
5   1.416639    0.044546
6   1.418157    0.032516
7   1.278280    0.051656

In [17]:
import pyneb as pn

sii = pn.Atom('S', 2)
Te = 10000
ne1 = sii.getTemDen(int_ratio=1.300890, tem=Te, wave1=6716, wave2=6731)
ne3 = sii.getTemDen(int_ratio=1.345630, tem=Te, wave1=6716, wave2=6731)
ne7 = sii.getTemDen(int_ratio=1.570315, tem=Te, wave1=6716, wave2=6731)
ne9 = sii.getTemDen(int_ratio=1.306804, tem=Te, wave1=6716, wave2=6731)
ne11 = sii.getTemDen(int_ratio=1.314939, tem=Te, wave1=6716, wave2=6731)
ne17 = sii.getTemDen(int_ratio=1.416639, tem=Te, wave1=6716, wave2=6731)
ne19 = sii.getTemDen(int_ratio=1.418157, tem=Te, wave1=6716, wave2=6731)
ne21 = sii.getTemDen(int_ratio=1.278280, tem=Te, wave1=6716, wave2=6731)

print(f"Densidad electrónica del slit 1: {ne1} cm^-3")
print(f"Densidad electrónica del slit 3: {ne3} cm^-3")
print(f"Densidad electrónica del slit 7: {ne7} cm^-3")
print(f"Densidad electrónica del slit 9: {ne9} cm^-3")
print(f"Densidad electrónica del slit 11: {ne11} cm^-3")
print(f"Densidad electrónica del slit 17: {ne17} cm^-3")
print(f"Densidad electrónica del slit 19: {ne19} cm^-3")
print(f"Densidad electrónica del slit 21: {ne21} cm^-3")





Densidad electrónica del slit 1: 157.53754255768436 cm^-3
Densidad electrónica del slit 3: 105.96813328201492 cm^-3
Densidad electrónica del slit 7: nan cm^-3
Densidad electrónica del slit 9: 150.78533191282756 cm^-3
Densidad electrónica del slit 11: 140.4512225354641 cm^-3
Densidad electrónica del slit 17: 34.75542408672053 cm^-3
Densidad electrónica del slit 19: 33.265773171295514 cm^-3
Densidad electrónica del slit 21: 185.17384589132743 cm^-3


In [3]:
def abundance_to_metallicity(O_H_log):
    # Abundancia de oxígeno a fracción de masa de oxígeno
    O_H = 10 ** (O_H_log - 12)
    X_O = O_H * (16 / 1) / (1 + 4 * O_H)
    
    # Fracción de masa de oxígeno a metalicidad total
    Z = X_O / 0.45
    return Z

O_H_log_1 = 8.040375
O_H_log_3 = 8.071898
O_H_log_7 = 8.178698
O_H_log_9 = 8.099142
O_H_log_11 = 8.171098
O_H_log_17 = 8.184468
O_H_log_19 = 8.082148
O_H_log_21 = 8.241325

O_H_log_1_N2 = 8.023009
O_H_log_3_N2 = 8.069726
O_H_log_7_N2 = 8.195424
O_H_log_9_N2 = 8.100274
O_H_log_11_N2 = 8.156117
O_H_log_17_N2 = 8.187604
O_H_log_19_N2 = 8.063999
O_H_log_21_N2 = 8.220582

Z1 = abundance_to_metallicity(O_H_log_1)
Z3 = abundance_to_metallicity(O_H_log_3)
Z7 = abundance_to_metallicity(O_H_log_7)
Z9 = abundance_to_metallicity(O_H_log_9)
Z11 = abundance_to_metallicity(O_H_log_11)
Z17 = abundance_to_metallicity(O_H_log_17)
Z19 = abundance_to_metallicity(O_H_log_19)
Z21 = abundance_to_metallicity(O_H_log_21)

Z1_N2 = abundance_to_metallicity(O_H_log_1_N2)
Z3_N2 = abundance_to_metallicity(O_H_log_3_N2)
Z7_N2 = abundance_to_metallicity(O_H_log_7_N2)
Z9_N2 = abundance_to_metallicity(O_H_log_9_N2)
Z11_N2 = abundance_to_metallicity(O_H_log_11_N2)
Z17_N2 = abundance_to_metallicity(O_H_log_17_N2)
Z19_N2 = abundance_to_metallicity(O_H_log_19_N2)
Z21_N2 = abundance_to_metallicity(O_H_log_21_N2)

Zsun = abundance_to_metallicity(8.69)
print(f'Zsun = {Zsun}')
print(f'Metalicidad Z para slit 1 con O3N2: {Z1} y con N2: {Z1_N2}')
print(f'Metalicidad Z para slit 3 con O3N2: {Z3} y con N2: {Z3_N2}')
print(f'Metalicidad Z para slit 7 con O3N2: {Z7} y con N2: {Z7_N2}')
print(f'Metalicidad Z para slit 9 con O3N2: {Z9} y con N2: {Z9_N2}')
print(f'Metalicidad Z para slit 11 con O3N2: {Z11} y con N2: {Z11_N2}')
print(f'Metalicidad Z para slit 17 con O3N2: {Z17} y con N2: {Z17_N2}')
print(f'Metalicidad Z para slit 19 con O3N2: {Z19} y con N2: {Z19_N2}')
print(f'Metalicidad Z para slit 21 con O3N2: {Z21} y con N2: {Z21_N2}')

Zsun = 0.017380307995068837
Metalicidad Z para slit 1 con O3N2: 0.0039002448170196054 y con N2: 0.0037474283495793612
Metalicidad Z para slit 3 con O3N2: 0.004193730544087258 y con N2: 0.0041728189778958156
Metalicidad Z para slit 7 con O3N2: 0.0053622048672883746 y con N2: 0.005572615978892059
Metalicidad Z para slit 9 con O3N2: 0.004465100697553733 y con N2: 0.004476748412012046
Metalicidad Z para slit 11 con O3N2: 0.0052692396040171065 y con N2: 0.005090678575151722
Metalicidad Z para slit 17 con O3N2: 0.0054338781883146794 y con N2: 0.005473233475917842
Metalicidad Z para slit 19 con O3N2: 0.004293837744701781 y con N2: 0.0041181789628486395
Metalicidad Z para slit 21 con O3N2: 0.006193409059339267 y con N2: 0.005904741346951046


In [9]:
'''
O_H_log_1 = 8.040375
O_H_log_3 = 8.071898
O_H_log_7 = 8.178698
O_H_log_9 = 8.099142
O_H_log_11 = 8.171098
O_H_log_17 = 8.184468
O_H_log_19 = 8.082148
O_H_log_21 = 8.241325

O_H_log_1_N2 = 8.023009
O_H_log_3_N2 = 8.069726
O_H_log_7_N2 = 8.195424
O_H_log_9_N2 = 8.100274
O_H_log_11_N2 = 8.156117
O_H_log_17_N2 = 8.187604
O_H_log_19_N2 = 8.063999
O_H_log_21_N2 = 8.220582
'''
𝑍1=10**(8.040375 - 8.69) * 0.02
𝑍3=10**(8.071898 - 8.69) * 0.02
𝑍7=10**(8.178698 - 8.69) * 0.02
𝑍9=10**(8.099142 - 8.69) * 0.02
𝑍11=10**(8.171098 - 8.69) * 0.02
𝑍17=10**(8.184468 - 8.69) * 0.02
𝑍19=10**(8.082148 - 8.69) * 0.02
𝑍21=10**(8.241325 - 8.69) * 0.02

Z1_N2 = 10**(8.023009 - 8.69) * 0.02
Z3_N2 = 10**(8.069726 - 8.69) * 0.02
Z7_N2 = 10**(8.195424 - 8.69) * 0.02
Z9_N2 = 10**(8.100274 - 8.69) * 0.02
Z11_N2 = 10**(8.156117 - 8.69) * 0.02
Z17_N2 = 10**(8.187604 - 8.69) * 0.02
Z19_N2 = 10**(8.063999- 8.69) * 0.02
Z21_N2 = 10**(8.220582 - 8.69) * 0.02

print(f'Metalicidad Z para slit 1 con O3N2: {Z1} y con N2: {Z1_N2}')
print(f'Metalicidad Z para slit 3 con O3N2: {Z3} y con N2: {Z3_N2}')
print(f'Metalicidad Z para slit 7 con O3N2: {Z7} y con N2: {Z7_N2}')
print(f'Metalicidad Z para slit 9 con O3N2: {Z9} y con N2: {Z9_N2}')
print(f'Metalicidad Z para slit 11 con O3N2: {Z11} y con N2: {Z11_N2}')
print(f'Metalicidad Z para slit 17 con O3N2: {Z17} y con N2: {Z17_N2}')
print(f'Metalicidad Z para slit 19 con O3N2: {Z19} y con N2: {Z19_N2}')
print(f'Metalicidad Z para slit 21 con O3N2: {Z21} y con N2: {Z21_N2}')


Metalicidad Z para slit 1 con O3N2: 0.004481310081202388 y con N2: 0.004305652695709633
Metalicidad Z para slit 3 con O3N2: 0.004818678991783061 y con N2: 0.004794639905410264
Metalicidad Z para slit 7 con O3N2: 0.0061620894122637075 y con N2: 0.0064040394069232644
Metalicidad Z para slit 9 con O3N2: 0.005130645350958532 y con N2: 0.0051440359570337595
Metalicidad Z para slit 11 con O3N2: 0.006055193076587442 y con N2: 0.005849880515597862
Metalicidad Z para slit 17 con O3N2: 0.00624450468338628 y con N2: 0.006289758861507125
Metalicidad Z para slit 19 con O3N2: 0.004933759727452544 y con N2: 0.004731828499521222
Metalicidad Z para slit 21 con O3N2: 0.007117951027380973 y con N2: 0.006785970942470441


In [8]:
import numpy as np

def logOH_to_Z(logOH):
    # Constantes
    atomic_mass_O = 16.0  # Masa atómica del oxígeno
    atomic_mass_H = 1.0   # Masa atómica del hidrógeno
    
    # Calcular O/H a partir de 12 + log(O/H)
    OH = 10**(logOH - 12)
    
    # Calcular la fracción de masa de oxígeno
    XO = (OH * atomic_mass_O) / (atomic_mass_H + OH * atomic_mass_O)
    
    # Calcular Z usando la aproximación Z ≈ 2 * X_O
    Z = 2 * XO
    
    return Z

#ejemplo sol
logOH_solar = 8.69
Z_solar = logOH_to_Z(logOH_solar)

print(f"Para 12 + log(O/H) = {logOH_solar}, la metalicidad Z es aproximadamente {Z_solar:.4f}")

# Puedes probar con otros valores de 12 + log(O/H)
O_H_log_1 = 8.040375
O_H_log_3 = 8.071898
O_H_log_7 = 8.178698
O_H_log_9 = 8.099142
O_H_log_11 = 8.171098
O_H_log_17 = 8.184468
O_H_log_19 = 8.082148
O_H_log_21 = 8.241325

Z_1 = logOH_to_Z(O_H_log_1)
Z_3 = logOH_to_Z(O_H_log_3)
Z_7 = logOH_to_Z(O_H_log_7)
Z_9 = logOH_to_Z(O_H_log_9)
Z_11 = logOH_to_Z(O_H_log_11)
Z_17 = logOH_to_Z(O_H_log_17)
Z_19 = logOH_to_Z(O_H_log_19)
Z_21 = logOH_to_Z(O_H_log_21)

O_H_log_1_N2 = 8.023009
O_H_log_3_N2 = 8.069726
O_H_log_7_N2 = 8.195424
O_H_log_9_N2 = 8.100274
O_H_log_11_N2 = 8.156117
O_H_log_17_N2 = 8.187604
O_H_log_19_N2 = 8.063999
O_H_log_21_N2 = 8.220582

Z_1_N2 = logOH_to_Z(O_H_log_1_N2)
Z_3_N2 = logOH_to_Z(O_H_log_3_N2)
Z_7_N2 = logOH_to_Z(O_H_log_7_N2)
Z_9_N2 = logOH_to_Z(O_H_log_9_N2)
Z_11_N2 = logOH_to_Z(O_H_log_11_N2)
Z_17_N2 = logOH_to_Z(O_H_log_17_N2)
Z_19_N2 = logOH_to_Z(O_H_log_19_N2)
Z_21_N2 = logOH_to_Z(O_H_log_21_N2)


print(f"Para slit 1: 12 + log(O/H) = {O_H_log_1} con O3N2, la metalicidad Z es aproximadamente {Z_1:.4f}")
print(f"Para slit 3: 12 + log(O/H) = {O_H_log_3} con O3N2, la metalicidad Z es aproximadamente {Z_3:.4f}")
print(f"Para slit 7: 12 + log(O/H) = {O_H_log_7} con O3N2, la metalicidad Z es aproximadamente {Z_7:.4f}")
print(f"Para slit 9: 12 + log(O/H) = {O_H_log_9} con O3N2, la metalicidad Z es aproximadamente {Z_9:.4f}")
print(f"Para slit 11: 12 + log(O/H) = {O_H_log_11} con O3N2, la metalicidad Z es aproximadamente {Z_11:.4f}")
print(f"Para slit 17: 12 + log(O/H) = {O_H_log_17} con O3N2, la metalicidad Z es aproximadamente {Z_17:.4f}")
print(f"Para slit 19: 12 + log(O/H) = {O_H_log_19} con O3N2, la metalicidad Z es aproximadamente {Z_19:.4f}")
print(f"Para slit 21: 12 + log(O/H) = {O_H_log_21} con O3N2, la metalicidad Z es aproximadamente {Z_21:.4f}")

Para 12 + log(O/H) = 8.69, la metalicidad Z es aproximadamente 0.0156
Para slit 1: 12 + log(O/H) = 8.040375 con O3N2, la metalicidad Z es aproximadamente 0.0035
Para slit 3: 12 + log(O/H) = 8.071898 con O3N2, la metalicidad Z es aproximadamente 0.0038
Para slit 7: 12 + log(O/H) = 8.178698 con O3N2, la metalicidad Z es aproximadamente 0.0048
Para slit 9: 12 + log(O/H) = 8.099142 con O3N2, la metalicidad Z es aproximadamente 0.0040
Para slit 11: 12 + log(O/H) = 8.171098 con O3N2, la metalicidad Z es aproximadamente 0.0047
Para slit 17: 12 + log(O/H) = 8.184468 con O3N2, la metalicidad Z es aproximadamente 0.0049
Para slit 19: 12 + log(O/H) = 8.082148 con O3N2, la metalicidad Z es aproximadamente 0.0039
Para slit 21: 12 + log(O/H) = 8.241325 con O3N2, la metalicidad Z es aproximadamente 0.0056


In [10]:
'''
O_H_log_1 = 8.040375
O_H_log_3 = 8.071898
O_H_log_7 = 8.178698
O_H_log_9 = 8.099142
O_H_log_11 = 8.171098
O_H_log_17 = 8.184468
O_H_log_19 = 8.082148
O_H_log_21 = 8.241325

O_H_log_1_N2 = 8.023009
O_H_log_3_N2 = 8.069726
O_H_log_7_N2 = 8.195424
O_H_log_9_N2 = 8.100274
O_H_log_11_N2 = 8.156117
O_H_log_17_N2 = 8.187604
O_H_log_19_N2 = 8.063999
O_H_log_21_N2 = 8.220582
'''
𝑍1=10**(8.040375 - 8.69) * 0.014
𝑍3=10**(8.071898 - 8.69) * 0.014
𝑍7=10**(8.178698 - 8.69) * 0.014
𝑍9=10**(8.099142 - 8.69) * 0.014
𝑍11=10**(8.171098 - 8.69) * 0.014
𝑍17=10**(8.184468 - 8.69) * 0.014
𝑍19=10**(8.082148 - 8.69) * 0.014
𝑍21=10**(8.241325 - 8.69) * 0.014

Z1_N2 = 10**(8.023009 - 8.69) * 0.014
Z3_N2 = 10**(8.069726 - 8.69) * 0.014
Z7_N2 = 10**(8.195424 - 8.69) * 0.014
Z9_N2 = 10**(8.100274 - 8.69) * 0.014
Z11_N2 = 10**(8.156117 - 8.69) * 0.014
Z17_N2 = 10**(8.187604 - 8.69) * 0.014
Z19_N2 = 10**(8.063999- 8.69) * 0.014
Z21_N2 = 10**(8.220582 - 8.69) * 0.014

print(f'Metalicidad Z para slit 1 con O3N2: {Z1} y con N2: {Z1_N2}')
print(f'Metalicidad Z para slit 3 con O3N2: {Z3} y con N2: {Z3_N2}')
print(f'Metalicidad Z para slit 7 con O3N2: {Z7} y con N2: {Z7_N2}')
print(f'Metalicidad Z para slit 9 con O3N2: {Z9} y con N2: {Z9_N2}')
print(f'Metalicidad Z para slit 11 con O3N2: {Z11} y con N2: {Z11_N2}')
print(f'Metalicidad Z para slit 17 con O3N2: {Z17} y con N2: {Z17_N2}')
print(f'Metalicidad Z para slit 19 con O3N2: {Z19} y con N2: {Z19_N2}')
print(f'Metalicidad Z para slit 21 con O3N2: {Z21} y con N2: {Z21_N2}')


Metalicidad Z para slit 1 con O3N2: 0.003136917056841672 y con N2: 0.0030139568869967433
Metalicidad Z para slit 3 con O3N2: 0.0033730752942481425 y con N2: 0.003356247933787185
Metalicidad Z para slit 7 con O3N2: 0.004313462588584595 y con N2: 0.004482827584846285
Metalicidad Z para slit 9 con O3N2: 0.0035914517456709723 y con N2: 0.0036008251699236314
Metalicidad Z para slit 11 con O3N2: 0.004238635153611209 y con N2: 0.004094916360918503
Metalicidad Z para slit 17 con O3N2: 0.004371153278370396 y con N2: 0.004402831203054987
Metalicidad Z para slit 19 con O3N2: 0.0034536318092167804 y con N2: 0.003312279949664855
Metalicidad Z para slit 21 con O3N2: 0.0049825657191666815 y con N2: 0.004750179659729309
